In [14]:
! pip install -q groq

In [16]:
from groq import Groq


client = Groq(api_key='')

ciao


In [17]:
#! pip install langchain
#! pip install langchain_groq
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0, groq_api_key='', model_name="llama3-70b-8192")

## Connecting to neo4j + Query Generation

In [18]:
! pip install neo4j
! pip install langchain
! pip install langchain_community

In [19]:
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="adminadmin"
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about data provenance.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Examples:
#Find all the entities
MATCH (n:Entity) RETURN n

#Find all the activities
MATCH (n:Activity) RETURN n

#Count all communities using louvain on WAS_DERIVED_FROM
CALL gds.graph.drop('proj', false)
YIELD graphName AS droppedGraph
WITH droppedGraph
CALL gds.graph.project('proj', ['Activity'], {{ WAS_DERIVED_FROM:{{orientation:'NATURAL'}} }})
YIELD graphName AS projectedGraph
WITH projectedGraph
CALL gds.louvain.mutate('proj', {{mutateProperty:'communityId'}})
YIELD communityCount
RETURN communityCount AS result

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"]
)


cypher_chain = GraphCypherQAChain.from_llm(
    llm=chat,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

print(graph.schema)

test = True
while test:
    print("\n\n Insert your question or type 'exit' to quit:")
    q = input("> ")
    if q == "exit":
        test = False
        break
    response = cypher_chain.invoke({"query": q})
    print(q)
    print(response["result"])

Node properties:
Activity {used_features: LIST, function_name: STRING, context: STRING, id: STRING, code: STRING}
Column {index: STRING, instance: STRING, value: STRING, id: STRING}
Relationship properties:

The relationships:
(:Activity)-[:USED]->(:Column)
(:Activity)-[:NEXT]->(:Activity)
(:Column)-[:WAS_INVALIDATED_BY]->(:Activity)
(:Column)-[:WAS_DERIVED_FROM]->(:Column)
(:Column)-[:WAS_GENERATED_BY]->(:Activity)


 Insert your question or type 'exit' to quit:


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n) RETURN count(n)
Full Context:
[{'count(n)': 75}]

> Finished chain.
return the number of nodes
The number of nodes is 75.


 Insert your question or type 'exit' to quit:


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Activity) RETURN count(n)
Full Context:
[{'count(n)': 8}]

> Finished chain.
return the number of activities
There are 8 activities.


 Insert your question or type 'exit' to quit:
